In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [4]:
device = torch.device("cpu")

In [5]:
model = Net().to(device)

In [45]:
for param in model.parameters():
    if len(param.size()) == 2 and param.size()[0] == 10 and param.size()[1] == 50:
        param.grad[9,:] = 0
        print(type(param.data), param.size(), param.grad[9,:])

<class 'torch.Tensor'> torch.Size([10, 50]) tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.])


In [7]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [9]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=1000, shuffle=True)

In [11]:
model.train()
for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)            

In [30]:
    loss = F.nll_loss(output, target)

In [31]:
    loss.backward()

In [35]:
model.parameters()

TypeError: 'generator' object has no attribute '__getitem__'

In [24]:
output[:, 9]

tensor([-2.2167, -2.1084, -2.2781, -2.2245, -2.3113, -2.2492, -1.9697,
        -2.1920, -2.3820, -2.2766, -2.3404, -2.4789, -2.3484, -2.1503,
        -2.1928, -2.2398, -1.9991, -2.2497, -2.3593, -2.0657, -2.2267,
        -2.1396, -2.4242, -2.4096, -2.4898, -2.2539, -2.0895, -2.2522,
        -2.2553, -2.2214, -2.2430, -2.1451])

In [28]:
type(torch.Tensor(len(output) * [0]).to(device))

torch.Tensor